# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [216]:
# Importing libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

In [217]:
# your code here
ds = pd.read_csv('Sentiment140.csv')
ds

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [218]:
# Getting a sample of the dataset as advised
ds_sample = ds.sample(20000)
ds_sample

,target,id,date,flag,user,text
1479792,4,2066735739,Sun Jun 07 11:04:35 PDT 2009,NO_QUERY,Brenda_Bullet,On the road 2 Rockin Roots!
1135062,4,1976442639,Sat May 30 17:20:27 PDT 2009,NO_QUERY,veganscore,@sarabethxvx It would be so SUPER sweet to kno...
262095,0,1986091203,Sun May 31 18:18:30 PDT 2009,NO_QUERY,superherogirl,@travisrail i may or may not have left my phon...
27830,0,1559384065,Sun Apr 19 11:00:02 PDT 2009,NO_QUERY,lovgloria,Waiting for the red LG Lotus to come in stock ...
1494169,4,2069640702,Sun Jun 07 16:15:06 PDT 2009,NO_QUERY,lynnedupuis,Making Apple Berry PIE Mmmmm..
...,...,...,...,...,...,...
1317555,4,2014274360,Wed Jun 03 01:00:01 PDT 2009,NO_QUERY,normaltusker,@thenokiablog u seem to be on a roll on ur vac...
509623,0,2189456489,Tue Jun 16 00:04:25 PDT 2009,NO_QUERY,abbigshmail,goodbye record stores. http://bit.ly/nKxt9
614028,0,2225697579,Thu Jun 18 11:02:21 PDT 2009,NO_QUERY,AnnaAura,almost freaking out cause of this damn headache
1581244,4,2190162132,Tue Jun 16 01:56:52 PDT 2009,NO_QUERY,NorthInvest_HB,"just had my morning coffee, ready to make some..."


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [219]:
# Functions from previous challenge
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    return re.sub(r'  *', ' ', re.sub(r'[^a-z]', ' ', 
                                      re.sub(r'www\.\S*', ' ', re.sub(r'http[s]?://\S*', ' ', s.lower())))).strip()


def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)


def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()
    
    stemmed_list = [ps.stem(w) for w in l]
    lemmed_on_stemmed_list = [wnl.lemmatize(w) for w in stemmed_list]
    
    return lemmed_on_stemmed_list


def remove_stopwords(lst, lang = 'english'):
    """
    Remove English (default) stopwords from a list of strings.

    Args:
        lst: A list of strings.
        l = Language of the stopwords.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = stopwords.words(lang)
    return [word for word in lst if word not in stop_words]

In [220]:
# your code here
#ds_sample['text_processed'] = ds_sample['text'].apply(clean_up)
#ds_sample['text_processed'] = ds_sample['text_processed'].apply(tokenize)
#ds_sample['text_processed'] = ds_sample['text_processed'].apply(stem_and_lemmatize)
#ds_sample['text_processed'] = ds_sample['text_processed'].apply(remove_stopwords)

# Looks like it worked. Can I do all 4 of these in the same line? Saw some people doing it but I didn't get it.
## some did in reverse order like 1st remove_stopwords and going back?
### might've answered myself when I rewrote the clean_up func before

In [221]:
# Ok, I decided to try and it worked :D
## I'll still take the explanation though :3
ds_sample['text_processed'] = \
ds_sample['text'].apply(lambda x: remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x)))))
ds_sample

,target,id,date,flag,user,text,text_processed
1479792,4,2066735739,Sun Jun 07 11:04:35 PDT 2009,NO_QUERY,Brenda_Bullet,On the road 2 Rockin Roots!,"[road, rockin, root]"
1135062,4,1976442639,Sat May 30 17:20:27 PDT 2009,NO_QUERY,veganscore,@sarabethxvx It would be so SUPER sweet to kno...,"[sarabethxvx, would, super, sweet, know, day, ..."
262095,0,1986091203,Sun May 31 18:18:30 PDT 2009,NO_QUERY,superherogirl,@travisrail i may or may not have left my phon...,"[travisrail, may, may, left, phone, car, accid..."
27830,0,1559384065,Sun Apr 19 11:00:02 PDT 2009,NO_QUERY,lovgloria,Waiting for the red LG Lotus to come in stock ...,"[wait, red, lg, lotu, come, stock, back, order]"
1494169,4,2069640702,Sun Jun 07 16:15:06 PDT 2009,NO_QUERY,lynnedupuis,Making Apple Berry PIE Mmmmm..,"[make, appl, berri, pie, mmmmm]"
...,...,...,...,...,...,...,...
1317555,4,2014274360,Wed Jun 03 01:00:01 PDT 2009,NO_QUERY,normaltusker,@thenokiablog u seem to be on a roll on ur vac...,"[thenokiablog, u, seem, roll, ur, vacat, mani,..."
509623,0,2189456489,Tue Jun 16 00:04:25 PDT 2009,NO_QUERY,abbigshmail,goodbye record stores. http://bit.ly/nKxt9,"[goodby, record, store]"
614028,0,2225697579,Thu Jun 18 11:02:21 PDT 2009,NO_QUERY,AnnaAura,almost freaking out cause of this damn headache,"[almost, freak, caus, thi, damn, headach]"
1581244,4,2190162132,Tue Jun 16 01:56:52 PDT 2009,NO_QUERY,NorthInvest_HB,"just had my morning coffee, ready to make some...","[morn, coffe, readi, make, money]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [222]:
bow = [word for lst in ds_sample['text_processed'] for word in lst]
fdist = FreqDist(bow)
most_common = fdist.most_common(5000)  # [w[0] for w in fdist.most_common(5000)] in case I just want the words
most_common

[('go', 1761),
 ('get', 1389),
 ('day', 1386),
 ('wa', 1296),
 ('good', 1131),
 ('thi', 1123),
 ('work', 1099),
 ('love', 1075),
 ('like', 1051),
 ('u', 915),
 ('quot', 913),
 ('got', 859),
 ('time', 832),
 ('today', 775),
 ('one', 768),
 ('thank', 755),
 ('lol', 746),
 ('want', 743),
 ('back', 675),
 ('see', 672),
 ('know', 660),
 ('miss', 645),
 ('feel', 627),
 ('realli', 621),
 ('think', 621),
 ('amp', 613),
 ('im', 576),
 ('well', 565),
 ('still', 555),
 ('hope', 554),
 ('make', 550),
 ('watch', 546),
 ('new', 544),
 ('need', 537),
 ('night', 536),
 ('home', 516),
 ('na', 507),
 ('ha', 497),
 ('come', 496),
 ('look', 492),
 ('oh', 476),
 ('much', 464),
 ('twitter', 439),
 ('wish', 438),
 ('great', 426),
 ('last', 425),
 ('morn', 420),
 ('tomorrow', 420),
 ('haha', 417),
 ('tri', 408),
 ('wait', 396),
 ('sleep', 393),
 ('sad', 374),
 ('happi', 363),
 ('whi', 362),
 ('onli', 357),
 ('week', 354),
 ('right', 353),
 ('bad', 347),
 ('fun', 344),
 ('would', 342),
 ('sorri', 338),
 ('foll

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [223]:
# your code here
# Mattia's
def find_features(document):
    words = set(document)
    features = {}
    for w in most_common: # word_features
        features[w] = w in words  # tried a few changes here, but output seems the same
    return features

                    #ds_sample['text_processed'] was originally here instead of just the 'text' one
documents = list(zip(ds_sample['text'], np.where(ds_sample['target'] == 4, True, False)))

# The output isn't exactly what we want, I think
featuresets = [(find_features(rev)) for (rev, category) in documents]  # tried few changes but output still not good

In [224]:
featuresets[0]

{('go', 1761): False,
 ('get', 1389): False,
 ('day', 1386): False,
 ('wa', 1296): False,
 ('good', 1131): False,
 ('thi', 1123): False,
 ('work', 1099): False,
 ('love', 1075): False,
 ('like', 1051): False,
 ('u', 915): False,
 ('quot', 913): False,
 ('got', 859): False,
 ('time', 832): False,
 ('today', 775): False,
 ('one', 768): False,
 ('thank', 755): False,
 ('lol', 746): False,
 ('want', 743): False,
 ('back', 675): False,
 ('see', 672): False,
 ('know', 660): False,
 ('miss', 645): False,
 ('feel', 627): False,
 ('realli', 621): False,
 ('think', 621): False,
 ('amp', 613): False,
 ('im', 576): False,
 ('well', 565): False,
 ('still', 555): False,
 ('hope', 554): False,
 ('make', 550): False,
 ('watch', 546): False,
 ('new', 544): False,
 ('need', 537): False,
 ('night', 536): False,
 ('home', 516): False,
 ('na', 507): False,
 ('ha', 497): False,
 ('come', 496): False,
 ('look', 492): False,
 ('oh', 476): False,
 ('much', 464): False,
 ('twitter', 439): False,
 ('wish', 438):

In [225]:
# your code here
# Jo's version seems to be the correct implementation of what's asked in this exercise, I believe
def find_features(document, bow):
    text = document.lower()
    features = dict()
    for w, c in bow:
        features[w] = w in text
    return features

def make_matrix(series_text, series_target, bow):
    return [(find_features(s, bow), bool(t)) for s, t in zip(series_text.values, series_target.values)]

matrix = make_matrix(ds_sample["text"], ds_sample["target"], most_common)

In [226]:
matrix[0] # leaving it like this [0] otherwise it makes it impossible to do anything as it becomes too heavy

({'go': False,
  'get': False,
  'day': False,
  'wa': False,
  'good': False,
  'thi': False,
  'work': False,
  'love': False,
  'like': False,
  'u': False,
  'quot': False,
  'got': False,
  'time': False,
  'today': False,
  'one': False,
  'thank': False,
  'lol': False,
  'want': False,
  'back': False,
  'see': False,
  'know': False,
  'miss': False,
  'feel': False,
  'realli': False,
  'think': False,
  'amp': False,
  'im': False,
  'well': False,
  'still': False,
  'hope': False,
  'make': False,
  'watch': False,
  'new': False,
  'need': False,
  'night': False,
  'home': False,
  'na': False,
  'ha': False,
  'come': False,
  'look': False,
  'oh': False,
  'much': False,
  'twitter': False,
  'wish': False,
  'great': False,
  'last': False,
  'morn': False,
  'tomorrow': False,
  'haha': False,
  'tri': False,
  'wait': False,
  'sleep': False,
  'sad': False,
  'happi': False,
  'whi': False,
  'onli': False,
  'week': False,
  'right': False,
  'bad': False,
  'fun

### Building and Training Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [227]:
# your code here

# Splitting into training and testing set
len(matrix) * 0.2  # 4000
training_set = matrix[:4000]
testing_set = matrix[4000:]

classifier = nltk.NaiveBayesClassifier.train(training_set)
classifier.show_most_informative_features(10)

Most Informative Features
                     sad = True            False : True   =     11.8 : 1.0
                 congrat = True             True : False  =     11.1 : 1.0
                  welcom = True             True : False  =      9.5 : 1.0
                    gone = True            False : True   =      8.9 : 1.0
                    sick = True            False : True   =      8.9 : 1.0
                    suck = True            False : True   =      8.5 : 1.0
                  forgot = True            False : True   =      7.3 : 1.0
                     yay = True             True : False  =      7.2 : 1.0
                   enjoy = True             True : False  =      7.1 : 1.0
                    babe = True             True : False  =      6.4 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [228]:
# your code here
print('Model accuracy:', nltk.classify.accuracy(classifier, testing_set))  # so bad :3

Model accuracy: 0.6784375


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [206]:
# your code here
# Increasing the sample size by 30k. Should help increase the accuray score
ds_sample2 = ds.sample(50000)
ds_sample2

,target,id,date,flag,user,text
467032,0,2175692656,Mon Jun 15 01:43:06 PDT 2009,NO_QUERY,lelloveschel,on msn.. looking at photos.. srsly need to tak...
904633,4,1694881479,Mon May 04 04:28:28 PDT 2009,NO_QUERY,MsGlamourista,@20somethingnl:20somethin' single
1154911,4,1978963085,Sun May 31 01:05:48 PDT 2009,NO_QUERY,embliss,I'm actually quite satisfied with the weather ...
437049,0,2065826044,Sun Jun 07 09:21:59 PDT 2009,NO_QUERY,toodamnninja,"@urbanfly lol already tried that one, same thi..."
193891,0,1970166512,Sat May 30 02:43:49 PDT 2009,NO_QUERY,utehbaik,"malaysia is just a place for layover, not wort..."
...,...,...,...,...,...,...
657931,0,2241252886,Fri Jun 19 10:43:33 PDT 2009,NO_QUERY,karynp86,so much work to do
371464,0,2050433578,Fri Jun 05 18:39:40 PDT 2009,NO_QUERY,Prettykeli,Still stuck on the sims 3 lol gotta take a bre...
655905,0,2240193173,Fri Jun 19 09:27:07 PDT 2009,NO_QUERY,myra_,"going home! ah, I don't want to leave Paris"
1595668,4,2192424390,Tue Jun 16 07:01:05 PDT 2009,NO_QUERY,MrsLarter,is beaming with pride over Brian's graduation ...


In [207]:
# Preparing the text for the analysis
ds_sample2['text_processed'] = \
ds_sample2['text'].apply(lambda x: remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x)))))
ds_sample2

,target,id,date,flag,user,text,text_processed
467032,0,2175692656,Mon Jun 15 01:43:06 PDT 2009,NO_QUERY,lelloveschel,on msn.. looking at photos.. srsly need to tak...,"[msn, look, photo, srsli, need, take, edit, im..."
904633,4,1694881479,Mon May 04 04:28:28 PDT 2009,NO_QUERY,MsGlamourista,@20somethingnl:20somethin' single,"[somethingnl, somethin, singl]"
1154911,4,1978963085,Sun May 31 01:05:48 PDT 2009,NO_QUERY,embliss,I'm actually quite satisfied with the weather ...,"[actual, quit, satisfi, weather, today, breakf..."
437049,0,2065826044,Sun Jun 07 09:21:59 PDT 2009,NO_QUERY,toodamnninja,"@urbanfly lol already tried that one, same thi...","[urbanfli, lol, alreadi, tri, one, thing, want..."
193891,0,1970166512,Sat May 30 02:43:49 PDT 2009,NO_QUERY,utehbaik,"malaysia is just a place for layover, not wort...","[malaysia, place, layov, worthi, show, first, ..."
...,...,...,...,...,...,...,...
657931,0,2241252886,Fri Jun 19 10:43:33 PDT 2009,NO_QUERY,karynp86,so much work to do,"[much, work]"
371464,0,2050433578,Fri Jun 05 18:39:40 PDT 2009,NO_QUERY,Prettykeli,Still stuck on the sims 3 lol gotta take a bre...,"[still, stuck, sim, lol, got, ta, take, break,..."
655905,0,2240193173,Fri Jun 19 09:27:07 PDT 2009,NO_QUERY,myra_,"going home! ah, I don't want to leave Paris","[go, home, ah, want, leav, pari]"
1595668,4,2192424390,Tue Jun 16 07:01:05 PDT 2009,NO_QUERY,MrsLarter,is beaming with pride over Brian's graduation ...,"[beam, pride, brian, graduat, today, good, job..."


In [209]:
# Creating the bag of words
bow = [word for lst in ds_sample2['text_processed'] for word in lst]
fdist = FreqDist(bow)

# Reducing the top feature number to train the model faster. Don't know if this increases the score or not
most_common = fdist.most_common(3000)
most_common

[('go', 4344),
 ('get', 3478),
 ('day', 3446),
 ('wa', 3402),
 ('thi', 2938),
 ('good', 2833),
 ('work', 2785),
 ('like', 2603),
 ('love', 2596),
 ('got', 2231),
 ('u', 2231),
 ('quot', 2206),
 ('today', 2165),
 ('time', 2077),
 ('lol', 1889),
 ('thank', 1878),
 ('one', 1842),
 ('miss', 1833),
 ('back', 1832),
 ('want', 1782),
 ('know', 1760),
 ('im', 1657),
 ('think', 1643),
 ('feel', 1630),
 ('see', 1614),
 ('realli', 1584),
 ('night', 1466),
 ('amp', 1454),
 ('hope', 1375),
 ('still', 1357),
 ('watch', 1351),
 ('need', 1350),
 ('new', 1340),
 ('make', 1333),
 ('well', 1325),
 ('na', 1318),
 ('home', 1287),
 ('ha', 1268),
 ('look', 1249),
 ('oh', 1248),
 ('come', 1193),
 ('twitter', 1163),
 ('much', 1151),
 ('last', 1147),
 ('morn', 1112),
 ('tomorrow', 1046),
 ('wish', 1029),
 ('great', 1028),
 ('sleep', 1027),
 ('haha', 1002),
 ('wait', 988),
 ('sad', 927),
 ('whi', 926),
 ('tri', 916),
 ('week', 894),
 ('fun', 888),
 ('onli', 885),
 ('veri', 877),
 ('thing', 863),
 ('bad', 848),
 

In [211]:
# Building the features
def find_features(document, bow):
    text = document.lower()
    features = dict()
    for w, c in bow:
        features[w] = w in text
    return features

def make_matrix(series_text, series_target, bow):
    return [(find_features(s, bow), bool(t)) for s, t in zip(series_text.values, series_target.values)]

matrix = make_matrix(ds_sample2['text'], ds_sample2['target'], most_common)

In [214]:
# Testing the model
len(matrix) * 0.25  # 12500

# Increased the size of the training set to 25% of the data
training_set = matrix[:12500]
testing_set = matrix[12500:]

classifier = nltk.NaiveBayesClassifier.train(training_set)
classifier.show_most_informative_features(10)

Most Informative Features
                  welcom = True             True : False  =     22.0 : 1.0
                 congrat = True             True : False  =     21.2 : 1.0
                unfortun = True            False : True   =     18.9 : 1.0
                   worst = True            False : True   =     14.9 : 1.0
                   scare = True            False : True   =     14.2 : 1.0
                    argh = True            False : True   =     13.6 : 1.0
                 appreci = True             True : False  =     11.8 : 1.0
               goodnight = True             True : False  =     10.5 : 1.0
                     sad = True            False : True   =      9.6 : 1.0
                    wont = True            False : True   =      8.9 : 1.0


In [215]:
# This result at least was above 0.7
print('Model accuracy:', nltk.classify.accuracy(classifier, testing_set))

Model accuracy: 0.7137333333333333


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here